### push geotifs to single folder for upload to map-kurator server on paperspace

In [ ]:
import arcpy
import os
import csv
import pdb
import time
from glob import glob
import pdb
import shutil
from pathlib import Path

aerials_root_tdrive = r"T:\Maps\Maps_00\Aerial_Scanning_Project_00"
mk_tif_path = r"C:\workspace\mk_geotif"
aerial_dirs = []
for i in os.scandir(aerials_root_tdrive):
    if i.is_dir() and not i.name.startswith("_"):
        aerial_dirs.append(os.path.join(i.path, "_index"))
               
for i in aerial_dirs:
    arcpy.env.workspace = i    
    county_name_and_year = i.split(os.path.sep)[4]
    print(county_name_and_year)
    tif_name = os.path.join(i, county_name_and_year +"_index_mosaic_wgs84.tif")
    if arcpy.Exists(tif_name):
        print("    tif exists, so let's copy it")
        dst = os.path.join(mk_tif_path, county_name_and_year +"_index_mosaic_wgs84.tif")
        if not arcpy.Exists(dst):
            shutil.copy(tif_name, dst)
            print("    done copying")

## process deep learning results

In [2]:
import arcpy
import os
import csv
import pdb
import time
from glob import glob
import pdb
from pathlib import Path

aerials_root_tdrive = r"T:\Maps\Maps_00\Aerial_Scanning_Project_00"
dl_model_path = r"C:\workspace\aerial_deep_learn_georef\RCNN_Masks_DLmodel2\RCNN_Masks_DLmodel2.emd"

proj = arcpy.mp.ArcGISProject("CURRENT")
map_view = proj.activeView
map1 = proj.activeMap
def build_county_year_list(root_path):
    wm = arcpy.SpatialReference(3857)
    dirs = [r"T:\Maps\Maps_00\Aerial_Scanning_Project_00\Adair_1939\_index"]
    #dirs = []
    cy_list = []
    if len(dirs) == 0:
        for i in os.scandir(root_path):
            if i.is_dir() and not i.name.startswith("_"):
                dirs.append(os.path.join(i.path, "_index"))

    for i in dirs:
        county_name_and_year = i.split(os.path.sep)[4]
        #print(county_name_and_year)
        cy_list.append((county_name_and_year, i))
        
    return cy_list

# location of geojson outputted by map kurator
#mk_geoj = r"C:\workspace\mk_geotif\_30s\geojson"
mk_geoj = r"C:\workspace\mk_geotif\test_input_geojson"

aerial_dirs = build_county_year_list(aerials_root_tdrive)


def build_gdb_dir(cny_path):
    return os.path.join(cny_path, cny_path.split(os.path.sep)[4] + "_index.gdb")

# convert geojson from map kurator into a GDB feature class
# returns path to conversion results
def convert_geojson(county_name_and_year, cny_path, geojson_root):
    index_gdb_dir = build_gdb_dir(cny_path)
    arcpy.env.workspace = cny_path
    proj = arcpy.mp.ArcGISProject("CURRENT")
    map_view = proj.activeView
    map1 = proj.activeMap

    geoj_file = glob(os.path.join(geojson_root, 
                     county_name_and_year + "*.geojson"))
    ## reference to map kurator output geojson   
    if len(geoj_file) == 0:
        print("     geojson doesn't exist")
        return False
    
    #print("    geojson file:", geoj_file)

    objects_name =  county_name_and_year + "_idx_mk_out"
    objects_path = os.path.join(index_gdb_dir, objects_name)
    
    if not arcpy.Exists(objects_path):
        print("    convert geojson to gdb feature class ")
        arcpy.conversion.JSONToFeatures(geoj_file[0], objects_path)
        print("        done")

    return objects_path
    
         
for (county_name_and_year,cny_path) in aerial_dirs:
    objects_path = convert_geojson(county_name_and_year, cny_path, mk_geoj)
    index_gdb_dir = build_gdb_dir(cny_path)
    print(county_name_and_year)
    if not objects_path:
        continue
        
    if arcpy.Exists(objects_path):
        print("    check buffer")
        bfr_path = objects_path + "_bfr"
        if not arcpy.Exists(bfr_path):
            bfr_lyr = arcpy.analysis.PairwiseBuffer(objects_path, bfr_path , "125 Meters")
        print("        done")

    
    print("    add MinX to buffer")
    if len(arcpy.ListFields(bfr_path, "MinX")) == 0:
        print("        adding MinX to buffer")
        arcpy.management.CalculateGeometryAttributes(bfr_path, "MinX EXTENT_MIN_X",coordinate_system=arcpy.SpatialReference(3857))
    else:
        print("        buffer already has MinX")
    print("        done")

    print("    dissolve")
    dis_path = bfr_path + "_dis"
    if not arcpy.Exists(dis_path):
        print("        dissolving buffer")
        arcpy.analysis.PairwiseDissolve(
            in_features=bfr_path,
            out_feature_class=dis_path,
            dissolve_field=None,
            statistics_fields=None,
            multi_part="SINGLE_PART"Create
        )
    else:
        print("        dissolve already exists")
        map1.addDataFromPath(dis_path)
    print("        done")
    
    
    print("    add 'text' field to dissolve")
    if len(arcpy.ListFields(dis_path, "text")) == 0:
        print("        adding 'text' to dissolve")
        arcpy.management.AddField(dis_path, "text","TEXT")
    else:
        print("        dissolve already has 'text'")
    print("        done")
    
          

    print("    spatial join")
    sj_path = dis_path + "_sj"
    if not arcpy.Exists(sj_path):
        print("        spatial joining dissolve")
        arcpy.analysis.SpatialJoin(
                target_features=dis_path,
                join_features=bfr_path,
                out_feature_class=sj_path,
                join_operation="JOIN_ONE_TO_ONE",
                join_type="KEEP_ALL",
                match_option="INTERSECT",
                search_radius=None,
                distance_field_name="",
                match_fields=None
        )
    else:
          print("        spatial join already exists")
    print("        done")

    print("    grab the spatial join layer")
    sj_name = os.path.split(sj_path)[-1]
    sj_lyr_list = map1.listLayers(sj_name)
    if len(sj_lyr_list)==0:
        print("        need to add sj layer to map")
        sj_lyr = map1.addDataFromPath(sj_path)
    if len(sj_lyr_list)==1:
        print("        layer already on map")
        sj_lyr = sj_lyr_list[0]
    
    print("        done")
         
    print("    merge all text values into single value")
    uc = arcpy.da.UpdateCursor(dis_path,["OBJECTID", "text"], where_clause="text IS NULL")
          
    for row in uc:
        #print("        row", row[0])
        #print(sj_lyr)
        sj_lyr.definitionQuery = "TARGET_FID = " + str(row[0])
        
        #use MinX to try and order the text for each label from left to right
        sc = arcpy.da.SearchCursor(sj_lyr,['text_1','MinX','SHAPE@'],  sql_clause=(None,"ORDER BY MinX ASC"))
        text_in_ordered_list = []
        for roww in sc:
            if roww[0] and roww[0] != "":
                text_in_ordered_list.append(roww[0])
        text_str = "||".join(text_in_ordered_list)
        row[1] = text_str
        uc.updateRow(row)
        
    print("        done")
    del uc

          
    with arcpy.EnvManager(processorType="GPU", scratchWorkspace=r""):


        dl_objects_name = county_name_and_year + "_index_obj"
        dl_objects_path = os.path.join(index_gdb_dir, dl_objects_name)

        #replacing this as part of integrating the code in this section
        mk_labels_name = county_name_and_year + "_idx_mk_out_bfr_dis"
        mk_labels_path = os.path.join(index_gdb_dir, mk_labels_name)
        mk_labels_lyr = map1.addDataFromPath(mk_labels_path)
        
        if not arcpy.Exists(dl_objects_path):
            print("        start deep learning")
            mosaic_name=os.path.join(index_gdb_dir, county_name_and_year + "_index_mosaic")
            if arcpy.Exists(mosaic_name):
                arcpy.ia.DetectObjectsUsingDeepLearning(
                    in_raster=mosaic_name,
                    out_detected_objects=dl_objects_path,
                    in_model_definition=dl_model_path,
                    arguments="padding 56;batch_size 12;threshold 0.5;return_bboxes False;test_time_augmentation False;merge_policy mean;tile_size 224",
                    run_nms="NO_NMS",
                    confidence_score_field="Confidence",
                    class_value_field="Class",
                    max_overlap_ratio=0,
                    processing_mode="PROCESS_AS_MOSAICKED_IMAGE"
                )
        else:
            print("        index objects already exist")

        dl_bfr_path = dl_objects_path + "_bfr"
        
        if not arcpy.Exists(dl_bfr_path):
            arcpy.analysis.PairwiseBuffer(dl_objects_path, dl_bfr_path, "125 Meters")
        #print("        done")

        print("    dissolve")
        
        dl_dis_path = dl_bfr_path + "_dis"

        if not arcpy.Exists(dl_dis_path):
        
            arcpy.analysis.PairwiseDissolve(
                in_features=dl_bfr_path,
                out_feature_class=dl_dis_path,
                dissolve_field=None,
                statistics_fields=None,
                multi_part="SINGLE_PART"
            )

        dl_dis_lyr = map1.addDataFromPath(dl_dis_path)

        arcpy.management.SelectLayerByLocation(
            in_layer=dl_dis_lyr,
            overlap_type="INTERSECT",
            select_features=mk_labels_lyr,
            search_distance="100 Meters",
            selection_type="NEW_SELECTION",
            invert_spatial_relationship="NOT_INVERT"
        )
        
        arcpy.management.DeleteFeatures(dl_dis_lyr)

        arcpy.management.Append(
            inputs=dl_dis_lyr,
            target=mk_labels_lyr,
            schema_type="NO_TEST",
            field_mapping='img_coordinates "img_coordinates" true true false 2000000000 Text 0 0,First,#;text "text_MAX" true true false 255 Text 0 0,First,#',
            subtype="",
            expression="",
            match_fields=None,
            update_geometry="NOT_UPDATE_GEOMETRY"
        )


        
        print("    convert to points")
        pts_name = mk_labels_name + "_pt"
        pts_path = os.path.join(index_gdb_dir, pts_name)
        if not arcpy.Exists(pts_path):
    
            pts_lyr = arcpy.management.CreateFeatureclass(index_gdb_dir, pts_name, "POINT", mk_labels_lyr, "DISABLED", "DISABLED", arcpy.SpatialReference(3857), "", "0", "0", "0")
            sc = arcpy.da.SearchCursor(mk_labels_lyr, ['SHAPE@','text'])
            pts = []
            for c in sc:
                pt = arcpy.Point(c[0].extent.XMax, c[0].extent.YMax)
                pts.append([pt,c[1]])

            ic = arcpy.da.InsertCursor(pts_lyr,["SHAPE@", "text"])

            for p in pts:
                ic.insertRow(p)
            print("        done")


#                     arcpy.management.FeatureVerticesToPoints(
#                         in_features=labels_lyr,
#                         out_feature_class=labels_path + "_pt",
#                         point_location="MID"
#                     )

        pts_fields = arcpy.ListFields(pts_path)
        if "FocalLength" not in pts_fields:
            
            print("    add fields to points")
            arcpy.management.AddFields(
                in_table=mk_labels_path + "_pt",
                field_description=None,
                template="photo_center_template"
            )
            print("        done")

        print("    remove intermediate layers") 
        [map1.removeLayer(i) for i in map1.listLayers(county_name_and_year +"*")] 
        print("        done")
print("    done")

Adair_1939
    check buffer
        done
    add MinX to buffer
        buffer already has MinX
        done
    dissolve
        dissolve already exists
        done
    add 'text' field to dissolve
        dissolve already has 'text'
        done
    spatial join
        spatial join already exists
        done
    grab the spatial join layer
        need to add sj layer to map
        done
    merge all text values into single value
        done
        index objects already exist
    dissolve
    convert to points
        done
    add fields to points
        done
    remove intermediate layers
        done
    done


# Delete derivatives

In [16]:
import arcpy
import os
import csv
import pdb
import time
from glob import glob
import pdb
from pathlib import Path
import re

def build_county_year_list(root_path):
    wm = arcpy.SpatialReference(3857)
    #dirs = [r"T:\Maps\Maps_00\Aerial_Scanning_Project_00\Atoka_1962\_index"]
    dirs = []
    cy_list = []
    if len(dirs) == 0:
        for i in os.scandir(root_path):
            if i.is_dir() and not i.name.startswith("_"):
                dirs.append(os.path.join(i.path, "_index"))

    for i in dirs:
        county_name_and_year = i.split(os.path.sep)[4]
        #print(county_name_and_year)
        cy_list.append((county_name_and_year, i))
        
    return cy_list


def build_gdb_dir(cny_path):
    return os.path.join(cny_path, cny_path.split(os.path.sep)[4] + "_index.gdb")


def delete_derivatives(county_name_and_year):
    ds = arcpy.ListFeatureClasses(county_name_and_year + "*")
    if ds and len(ds)>0:
        print("         ", ds)
        ds = [i for i in ds if (i.endswith("bfr") or i.endswith("sj") or i.endswith("dis") or i.endswith("pts"))]
        #ds = ds + arcpy.ListFeatureClasses(county_name_and_year + "*bfr")   
        #ds = ds + arcpy.ListFeatureClasses(county_name_and_year + "*sj")
        #ds = ds + arcpy.ListFeatureClasses(county_name_and_year + "*pts")
        if len(ds) > 0:
            print(ds)
            print(arcpy.Delete_management(ds))

    
    
    
dirs = build_county_year_list(r"T:\Maps\Maps_00\Aerial_Scanning_Project_00")
#print(arcpy.env.workspace)
for (county_name_and_year, cny_path) in dirs:
    print("county_name_and_year: ", county_name_and_year)
    #print("cny_path: ", cny_path)
    gdb_dir = build_gdb_dir(cny_path)
    print(gdb_dir)
    if (os.path.exists(gdb_dir)):
        print("      found it")
        arcpy.env.workspace = gdb_dir
        delete_derivatives(county_name_and_year)


county_name_and_year:  Adair_1939
T:\Maps\Maps_00\Aerial_Scanning_Project_00\Adair_1939\_index\Adair_1939_index.gdb
      found it
          ['Adair_1939_idx_detect_obj', 'Adair_1939_index_obj']
county_name_and_year:  Adair_1964
T:\Maps\Maps_00\Aerial_Scanning_Project_00\Adair_1964\_index\Adair_1964_index.gdb
      found it
          ['Adair_1964_idx_mk_out', 'Adair_1964_index_obj']
county_name_and_year:  Alfalfa_1942
T:\Maps\Maps_00\Aerial_Scanning_Project_00\Alfalfa_1942\_index\Alfalfa_1942_index.gdb
      found it
          ['Alfalfa_1942_idx_mk_out']
county_name_and_year:  Alfalfa_1973
T:\Maps\Maps_00\Aerial_Scanning_Project_00\Alfalfa_1973\_index\Alfalfa_1973_index.gdb
      found it
          ['Alfalfa_1973_index_obj', 'Alfalfa_1973_index_obj_buffer', 'Alfalfa_1973_idx_mk_out']
county_name_and_year:  Atoka_1962
T:\Maps\Maps_00\Aerial_Scanning_Project_00\Atoka_1962\_index\Atoka_1962_index.gdb
      found it
          ['Atoka_1962_index_obj', 'Atoka_1962_idx_mk_out', 'Atoka_1962_id

      found it
          ['Kingfisher_1970_index_obj', 'Kingfisher_1970_index_obj_buffer']
county_name_and_year:  Kiowa_1937
T:\Maps\Maps_00\Aerial_Scanning_Project_00\Kiowa_1937\_index\Kiowa_1937_index.gdb
      found it
          ['Kiowa_1937_index_obj', 'Kiowa_1937_index_obj_buffer']
county_name_and_year:  Kiowa_1957
T:\Maps\Maps_00\Aerial_Scanning_Project_00\Kiowa_1957\_index\Kiowa_1957_index.gdb
      found it
          ['Kiowa_1957_index_obj', 'Kiowa_1957_index_obj_buffer']
county_name_and_year:  Kiowa_1964
T:\Maps\Maps_00\Aerial_Scanning_Project_00\Kiowa_1964\_index\Kiowa_1964_index.gdb
      found it
          ['Kiowa_1964_index_obj', 'Kiowa_1964_index_obj_buffer']
county_name_and_year:  Kiowa_1970
T:\Maps\Maps_00\Aerial_Scanning_Project_00\Kiowa_1970\_index\Kiowa_1970_index.gdb
      found it
          ['Kiowa_1970_index_obj', 'Kiowa_1970_index_obj_buffer']
county_name_and_year:  Latimer_1967
T:\Maps\Maps_00\Aerial_Scanning_Project_00\Latimer_1967\_index\Latimer_1967_index.gdb

T:\Maps\Maps_00\Aerial_Scanning_Project_00\Payne_1979\_index\Payne_1979_index.gdb
      found it
          ['Payne_1979_index_obj', 'Payne_1979_index_obj_buffer', 'Payne_1979_index_obj_buffer_sj', 'Payne_1979_index_obj_buffer_sj_dis']
['Payne_1979_index_obj_buffer_sj', 'Payne_1979_index_obj_buffer_sj_dis']
true
county_name_and_year:  Pittsburg_1967
T:\Maps\Maps_00\Aerial_Scanning_Project_00\Pittsburg_1967\_index\Pittsburg_1967_index.gdb
      found it
          ['Pittsburg_1967_index_obj', 'Pittsburg_1967_index_obj_buffer', 'Pittsburg_1967_index_obj_buffer_sj', 'Pittsburg_1967_index_obj_buffer_sj_dis']
['Pittsburg_1967_index_obj_buffer_sj', 'Pittsburg_1967_index_obj_buffer_sj_dis']
true
county_name_and_year:  Pontotoc_1938
T:\Maps\Maps_00\Aerial_Scanning_Project_00\Pontotoc_1938\_index\Pontotoc_1938_index.gdb
      found it
          ['Pontotoc_1938_index_obj', 'Pontotoc_1938_index_obj_buffer', 'Pontotoc_1938_index_obj_buffer_sj', 'Pontotoc_1938_index_obj_buffer_sj_dis']
['Pontotoc_193

In [ ]:

proj = arcpy.mp.ArcGISProject("CURRENT")
map_view = proj.activeView
map1 = proj.activeMap

layers = map1.listLayers("*idx_mk_out_bfr_dis_sj")
for lyr in layers:
    print(lyr.name)
    arcpy.management.DeleteField(lyr,["Join_Count",
        "TARGET_FID",
        "Join_Count_1",
        "TARGET_FID_1",
        "text",
        "score",
        "ORIG_FID",
        "text_Join",
        "Shape_Length_1",
        "Shape_Area_1"])
    print("    done")

# Extract GCPs

In [3]:
"""
Extract corner coordinates as "control points" using GDAL/Rasterio
"""


import rasterio
from rasterio import Affine
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.transform import from_origin
from rasterio.warp import transform_bounds
from rasterio.control import GroundControlPoint

import json
import csv
import os
import glob
from pprint import pprint

input_folder = r"C:\workspace\mk_geotif\ad\\"
input_files = glob.glob(os.path.join(input_folder, "*.tif"))
 
csv_file = open(os.path.join(input_folder,"output_GCPs.csv"), "w", newline="\n",encoding="utf8")
csv_writer = csv.DictWriter(csv_file,fieldnames={"external_id", "gcps"})
csv_writer.writeheader()

print("starting:")
for input_file in input_files:
    print("  now on:", input_file)
    input_raster = rasterio.open(input_file)
    input_file_name = os.path.split(input_file)[-1]
    height = input_raster.height
    width = input_raster.width

    bounds = input_raster.bounds
    l1 = bounds.left 
    r1 = bounds.right
    t1 = bounds.top
    b1 = bounds.bottom

    bounds_wgs84 = transform_bounds(input_raster.crs, 
                    rasterio.CRS.from_epsg(4326), 
                    l1, b1, r1, t1, 
                    densify_pts=21)
    #pprint(bounds_wgs84)
    
    l = bounds_wgs84[0] 
    b = bounds_wgs84[1]
    r = bounds_wgs84[2]
    t = bounds_wgs84[3]
                            #row, col, x, y
    tl = GroundControlPoint(0, 0, t, l)
    bl = GroundControlPoint(height, 0, b, l)
    br = GroundControlPoint(height, width, b, r)
    tr = GroundControlPoint(0, width, t, r)

    gcps = [tl, bl, br, tr]
    gcps_dicts = []
    #pprint(gcps)
    for gcp in gcps:
        print("[y,x]:",gcp.y,gcp.x)
        gcps_dicts.append({"pixel":[gcp.col,gcp.row],
            "location": [gcp.y,gcp.x ]               
            })

    #print(gcps_dicts)

    #this is happening on MK end, so don't need it here (yet....??)
    #gcp_str = ''
    #for gcp in gcps_dicts:
    #   lng, lat = gcp['location']
    #   x, y = gcp['pixel']
    #   gcp_str += '-gcp ' + str(x) + ' ' + str(y) + ' ' + str(lng) + ' ' + str(lat) + ' '
    
    #print(str(gcps_dicts))
    csv_writer.writerow({"external_id":os.path.splitext(input_file_name)[0], "gcps": str(gcps_dicts)})

csv_file.close()


starting:
  now on: C:\workspace\mk_geotif\ad\Adair_1939_index_mosaic_wgs84.tif
[y,x]: -94.86756698170343 36.23743258371726
[y,x]: -94.86756698170343 35.54002840219862
[y,x]: -94.42408674724588 35.54002840219862
[y,x]: -94.42408674724588 36.23743258371726
